SQL Tutorial

Connect database:
Database is on local machine and called Deneme. You need to specify your own user name and password.

In [1]:
%defaultDatasource jdbc:postgresql://localhost/Deneme?user=username&password=123456

In [110]:
-- drop table salary2017;

In [ ]:
-- create a table called salary2017 to store data
-- just run the code to execute, we will cover details later
CREATE TABLE salary2017(
    name varchar(40),
    gender char(1),
    annualSalary float,
    grossPayReceived2017 float,
    overtimePay2017 float,
    depCode char(4),
    divison varchar(100),
    empPositionTitle varchar(100),
    positionUnderFilled varchar(100),
    dateFirstHired date
);
-- dataset from https://data.montgomerycountymd.gov/Human-Resources/Employee-Salaries-2017
-- some modifications are performed on file in order to copy into our table
COPY salary2017 FROM 'C:\Users\zormu\Desktop\sql tutorial\2017Salary.csv' DELIMITERS ',' CSV HEADER;

In [ ]:
-- create a table called salary2017 to store data
-- just run the code to execute, we will cover details later
CREATE TABLE salary2018(
    name varchar(40),
    gender char(1),
    annualSalary float,
    grossPayReceived2017 float,
    overtimePay2017 float,
    depCode char(4),
    divison varchar(100),
    empPositionTitle varchar(100),
    positionUnderFilled varchar(100),
    dateFirstHired date
);
-- dataset from https://data.montgomerycountymd.gov/Human-Resources/Employee-Salaries-2018/kdqy-4wzv
-- some modifications are performed on file in order to copy into our table
COPY salary2018 FROM 'C:\Users\zormu\Desktop\sql tutorial\2018Salary.csv' DELIMITERS ',' CSV HEADER;

In [ ]:
-- Create Look-up table for department code and corresponding names
/*
CREATE TABLE depnames(
    code char(3),
    name varchar(100)
);
COPY depnames FROM 'C:\Users\zormu\Desktop\sql tutorial\depNames.csv' DELIMITERS ',' CSV HEADER;
CREATE TABLE dep_code_names
  AS (SELECT DISTINCT code,name FROM depnames);
*/

BASIC QUERIES

In [2]:
SELECT * FROM salary2017;               --Select all data from table

In [3]:
SELECT * FROM salary2017
AS A LIMIT 5;                      -- to keep notebook shorter, display the outcomes up to 5 entries

In [4]:
SELECT name,depcode,annualsalary        --Select some columns from table
FROM salary2017
AS A LIMIT 5;    

In [5]:
SELECT * FROM                                      --you may select all data from a table created by another SELECT statement 
(SELECT name,depcode,annualsalary FROM salary2017) --this part creates a table as above
AS A LIMIT 5;

In [6]:
SELECT name,depcode,annualsalary        --WHERE filters data
FROM salary2017                         --Operators: =  <  >  <=  >=  <>
WHERE depcode = 'POL';

In [7]:
SELECT name,depcode,annualsalary                  --multiple conditions
FROM salary2017                                   --Operators: AND OR NOT
WHERE depcode = 'POL' AND annualsalary >= 50000;

In [8]:
SELECT name,gender,datefirsthired        -- names starting with Ber
FROM salary2017                          -- % stands for multiple any characters
WHERE name LIKE 'Ber%';                  -- _ stands for single any character

In [9]:
SELECT name,gender,datefirsthired         -- BETWEEN is for numerical values
FROM salary2017                          
WHERE annualsalary BETWEEN 50000 AND 70000;                  

In [10]:
SELECT name,gender,annualsalary           -- IS NULL selects empty cells
FROM salary2017                           -- IS NOT NULL selects non-emoty cells
WHERE positionunderfilled IS NOT NULL;  

In [11]:
SELECT DISTINCT depcode FROM salary2017;      -- detects distinct values fom given column

In [12]:
SELECT name,annualsalary,depcode        
FROM salary2017                          
WHERE depcode IN ('POL','HHS','COR');    -- equivalent to WHERE depcode='POL' OR depcode='HHS' OR depcode='COR';

In [13]:
SELECT DISTINCT(depcode) FROM salary2017 -- use this resul as a list for WHERE IN in the following query
WHERE annualsalary > 220000;

In [14]:
SELECT name,annualsalary,depcode        
FROM salary2017                          
WHERE depcode IN(                  -- you may write another select in paranthesis to get values
SELECT DISTINCT(depcode) FROM salary2017 
WHERE annualsalary > 220000
);  

In [15]:
SELECT name,gender,annualsalary,depcode     -- order by depcode descending and then annualsalary ascending
FROM salary2017                          
ORDER BY depcode DESC,annualsalary;  

In [16]:
SELECT name,gender,annualsalary,depcode      -- first filter and then order
FROM salary2017
WHERE gender = 'F'
ORDER BY annualsalary DESC;  

Aggregate Functions: COUNT, SUM, AVG, MAX, MIN

In [17]:
SELECT COUNT(*) FROM salary2017;              -- counts number of entries in table

9398

In [18]:
SELECT COUNT(*) AS num_of_entries             -- AS renames output column, but not observable here
FROM salary2017;       

9398

In [19]:
SELECT COUNT(*) AS num_of_highSalary           -- number of high salaries
FROM salary2017
WHERE annualsalary >= 200000;       

20

In [20]:
SELECT 
MAX(annualsalary) AS max_salary,        
MIN (annualsalary) AS min_salary,
AVG (annualsalary) AS average_salary,
SUM (annualsalary) AS total_salary,
MIN(dateFirstHired) AS oldest_employee,
MAX(dateFirstHired) AS newest_employee
FROM salary2017;       

In [21]:
SELECT divison FROM salary2017                  -- groups data based on disivion, 
GROUP BY divison;                               -- in this case it is identical to DISTINCT

In [22]:
SELECT divison, COUNT(*) AS num_of_employee         -- counts total number of employees in each divison 
FROM salary2017                                  -- and list in descending order 
GROUP BY divison
ORDER BY num_of_employee DESC; 

In [23]:
SELECT divison, COUNT(*) AS num_of_employee         -- restriction on grouped data is conducted with HAVING 
FROM salary2017                                  
GROUP BY divison
HAVING divison LIKE 'M%'; 

In [24]:
SELECT divison, COUNT(*) AS num_of_employee        
FROM salary2017                                  
GROUP BY divison
HAVING COUNT(*) >= 70
ORDER BY num_of_employee DESC; 

In [25]:
SELECT divison, AVG(annualsalary) AS average_salary        -- average salary of each divison greater than 150.000
FROM salary2017                                  
GROUP BY divison
HAVING AVG(annualsalary) >= 150000
ORDER BY average_salary DESC; 

Query on Multiple Tables

In [26]:
SELECT COUNT(*) FROM salary2017;
SELECT COUNT(*) FROM salary2018;

In [27]:
-- SELECT * FROM salary2017, salary2018;     -- Selects from multiple tables, results all combinations,9378x10071 = 94.445.838 rows

In [28]:
SELECT COUNT(*) FROM salary2017, salary2018    -- counts matching names from both tables 
WHERE salary2017.name = salary2018.name;

9393

In [29]:
SELECT s7.name,
s7.annualsalary AS annualsalary2017, 
s8.annualsalary AS annualsalary2018                             --change same names from different tables
FROM salary2017 s7, salary2018 s8                               -- use alias to keep code clear
WHERE s7.name = s8.name;

In [30]:
SELECT s7.name,                                         -- INNER JOIN all matching values from left(s7) and right(s8) tables
s7.annualsalary AS annualsalary2017,                    -- exactly same with above query
s8.annualsalary AS annualsalary2018         
FROM salary2017 s7                                      
INNER JOIN salary2018 s8
ON s7.name = s8.name;

In [31]:
SELECT * FROM dep_code_names;

In [32]:
SELECT s7.name,                                         --twice INNER JOIN, add deparment names
s7.annualsalary AS annualsalary2017,                    
s8.annualsalary AS annualsalary2018,
s8.depcode, d.name AS departmentname   
FROM salary2017 s7                                       
INNER JOIN salary2018 s8 ON s7.name = s8.name
INNER JOIN dep_code_names d ON s8.depcode = d.code;

In [33]:
/*
-- create a small table
CREATE TABLE employee(
    id int,
    name varchar(40),
    depCode char(4),
    supervisor int
);
COPY employee FROM 'C:\Users\zormu\Desktop\sql tutorial\employee.csv' DELIMITERS ',' CSV HEADER;
*/
SELECT * FROM employee;   -- this table keeps some employees and id of their supervisors

In [34]:
--SELF JOIN, joining a table to itself
--use same table, but name as emp and sup to treate as 2 tables
SELECT emp.*, sup.name as name_of_supervisor
FROM employee emp        --employee table                                     
INNER JOIN employee sup  --supervisor table
ON emp.supervisor = sup.id;

In [35]:
SELECT s7.name,                                         
s7.annualsalary AS annualsalary2017,                    
s8.annualsalary AS annualsalary2018       
FROM salary2017 s7                                       
LEFT OUTER JOIN salary2018 s8
ON s7.name = s8.name;

In [36]:
SELECT s7.name,                                         
s7.annualsalary AS annualsalary2017,                    
s8.annualsalary AS annualsalary2018         
FROM salary2017 s7                                       
RIGHT OUTER JOIN salary2018 s8
ON s7.name = s8.name;

In [37]:
SELECT s7.name,                                         
s7.annualsalary AS annualsalary2017,                    
s8.annualsalary AS annualsalary2018         
FROM salary2017 s7                                       
FULL OUTER JOIN salary2018 s8
ON s7.name = s8.name;

In [38]:
/*
SELECT s7.name, s7.annualsalary, s8.annualsalary         
FROM salary2017 s7                                       
CROSS JOIN salary2018 s8;
*/

In [39]:
-- UNION of tables: columns order and types must be same
(SELECT * FROM salary2018 WHERE annualsalary > 220000)
UNION
(SELECT * FROM salary2018 WHERE annualsalary < 16000);

In [40]:
(SELECT * FROM salary2018 WHERE annualsalary > 220000)
UNION                                                     -- eliminates repeating entries
(SELECT * FROM salary2018 WHERE annualsalary > 220000);

In [41]:
(SELECT * FROM salary2018 WHERE annualsalary > 220000)
UNION ALL                                               -- allows repeating entries
(SELECT * FROM salary2018 WHERE annualsalary > 220000);

In [42]:
-- INTERSECT tables: columns order and types must be same
(SELECT name FROM salary2017 WHERE annualsalary > 220000)
INTERSECT
(SELECT name FROM salary2018 WHERE annualsalary > 220000);